# correlation

In [1]:
import warnings
warnings.simplefilter("ignore")
from scipy import stats
import numpy as np
import pandas as pd
#
import matplotlib
import matplotlib.pyplot as plt
#
import seaborn as sns

In [2]:
# 描画設定
plt.rcParams['figure.dpi'] = 300
plt.rcParams.update({'font.family': 'sans-serif', 'text.usetex': False,'pcolor.shading':'auto'})

In [3]:
class Site:
    def DomeF(self):
        # DomeF
        # 南緯77度19分01秒 東経39度42分12秒座標: 南緯77度19分01秒 東経39度42分12秒
        sx=14 ; sy=60 #;; # 77°18′59″S 39°42′04″E
        df_lat=-77.3
        df_lon=39.66
        return df_lon,df_lat,sx,sy


df_lon,df_lat,df_x,df_y = Site().DomeF()

In [4]:
def Mon():
    import pandas as pd
    mons = pd.DataFrame({
            "": ["Jan", "Feb", "Mar","Apr","May","Jun",
                     "Jul","Aug","Sep","Oct","Nov","Dec","Jan"],
            })
    return mons
mons=Mon()

In [5]:
loadfile = "T2.npz"
dataset = np.load(loadfile)
T2 = dataset["T2"]
lon2 = dataset["lon2"]
lat2 = dataset["lat2"]
y = dataset["y"]
m = dataset["m"]
d = dataset["d"]

In [6]:
loadfile = "prcp.npz"
dataset = np.load(loadfile)
prcp = dataset["prcp"]

In [7]:
loadfile = "prcp_d18O.npz"
dataset = np.load(loadfile)
prcp_d18O = dataset["prcp_d18O"]

In [8]:
loadfile = "sam.npz"
dataset = np.load(loadfile)
sam  = dataset["sam"][:]

In [9]:
temp_df = T2  [df_x-1, df_y-1, :]
prcp_df = prcp[df_x-1, df_y-1, :]
# removing abnormal values
is_r_prcp_d18O = np.squeeze(prcp_d18O[df_x-1, df_y-1,:]<-125)
prcp_d18O_df   = np.zeros(len(m))
prcp_d18O_df   = prcp_d18O[df_x-1, df_y-1,:]
prcp_d18O_df[is_r_prcp_d18O] = np.nan

# Model results

In [10]:
df_all = pd.DataFrame.from_dict({
    "Number": np.zeros((len(prcp_d18O_df))),
    "Year": y,
    "Mon" : m,
    "Date": d,
    "d18O": prcp_d18O_df,
    "temp": temp_df     ,
    "prcp": prcp_df     ,
    "sam" : sam         ,
},orient="columns")

In [11]:
df_all = df_all.dropna(how="any")
num = np.zeros(13)
for mm in range(12):
    num[mm] = len(df_all[df_all["Mon"]==mm+1])
num[12] =len(df_all)

## Standard deviations

In [12]:
def std(varin, var):
    df_out = pd.DataFrame.from_dict({
        "std":np.zeros(13)
    }, orient="columns")
    indexlist = mons[""][:12].tolist()
    indexlist.append("Annual")
    df_out.index = indexlist
    for mm in range(12):
        df =  varin[varin["Mon"]==mm+1]   
        df_out["std"][mm] = np.nanstd(df[var])
    df_out["std"][12] = np.nanstd(varin[var])

    return df_out.T

In [13]:
indexlist = mons[""][:12].tolist()
indexlist.append("Annual")

df_std = pd.DataFrame(data=[map(str,map(int,num)),
                             ['{:.2f}'.format(n) for n in (std(df_all[["Mon","temp"]], "temp").T["std"].values )],
                             ['{:.2f}'.format(n) for n in (std(df_all[["Mon","d18O"]], "d18O").T["std"].values )],
                             ['{:.2f}'.format(n) for n in (std(df_all[["Mon","prcp"]], "prcp").T["std"].values )],
          ], columns=indexlist,
    index=["Number","$\mathsf{\Delta SAT}$","$\mathsf{\Delta\delta^{18}O_p}$", "$\mathsf{P}$"])

display(df_std)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
Number,930,847,930,900,926,896,922,923,884,853,893,930,10834
$\mathsf{\Delta SAT}$,3.59,6.41,6.76,6.96,8.35,8.54,8.67,7.52,7.23,7.43,6.01,3.46,15.71
$\mathsf{\Delta\delta^{18}O_p}$,7.61,13.66,10.60,11.73,13.71,13.53,14.52,13.68,13.09,15.67,13.06,7.67,25.54
$\mathsf{P}$,0.25,0.16,0.11,0.15,0.15,0.20,0.15,0.10,0.09,0.31,0.18,0.29,0.19


In [14]:
prcp_summer = df_all[(df_all["Mon"]<=2)+(df_all["Mon"]>=11)]["prcp"]
prcp_winter = df_all[(df_all["Mon"]>=3)+(df_all["Mon"]<=10)]["prcp"]
stats.mannwhitneyu(prcp_winter, prcp_summer, use_continuity=True, alternative=None)

MannwhitneyuResult(statistic=18453555.0, pvalue=6.600122525722355e-07)

In [15]:
temp_summer = df_all[(df_all["Mon"]<=2)+(df_all["Mon"]>=10)]["temp"]
temp_winter = df_all[(df_all["Mon"]>=3)+(df_all["Mon"]<=9)]["temp"]
stats.bartlett(temp_winter, temp_summer)

BartlettResult(statistic=724.1172189225003, pvalue=1.7036552230148304e-159)

In [16]:
d18O_summer = df_all[(df_all["Mon"]<=2)+(df_all["Mon"]>=10)]["d18O"]
d18O_winter = df_all[(df_all["Mon"]>=3)+(df_all["Mon"]<=9)]["d18O"]
stats.bartlett(d18O_winter, d18O_summer)

BartlettResult(statistic=153.39170007615118, pvalue=3.145343801388662e-35)

# Observation

In [17]:
obs_DomeF = np.genfromtxt("obs_DomeF.csv",
                           delimiter=",", # 区切り文字
                           usecols=(0,1,2,3,4,5) # 読み込みたい列
                          )

obs_len          = np.shape(obs_DomeF)[0]
obs_y_df         = obs_DomeF[1:,0]
obs_m_df         = obs_DomeF[1:,1]
obs_d_df         = obs_DomeF[1:,2]
obs_temp_df      = obs_DomeF[1:,3]
obs_prcp_df      = obs_DomeF[1:,4]
obs_prcp_d18O_df = obs_DomeF[1:,5]

obs_dayst  = 31
obs_dayed  = 31 + 365
nanlist    = np.zeros((obs_dayed-obs_len+1))
nanlist[:] = np.nan

obs_y_df         = np.append(obs_y_df        [obs_dayst:obs_dayed], nanlist)
obs_m_df         = np.append(obs_m_df        [obs_dayst:obs_dayed], nanlist)
obs_d_df         = np.append(obs_d_df        [obs_dayst:obs_dayed], nanlist)
obs_temp_df      = np.append(obs_temp_df     [obs_dayst:obs_dayed], nanlist)
obs_prcp_df      = np.append(obs_prcp_df     [obs_dayst:obs_dayed], nanlist)
obs_prcp_d18O_df = np.append(obs_prcp_d18O_df[obs_dayst:obs_dayed], nanlist)

In [18]:
df_obs = pd.DataFrame.from_dict({
    "Number": np.zeros((len(obs_prcp_d18O_df))),
    "Year": obs_y_df,
    "Mon" : obs_m_df,
    "Date": obs_d_df,
    "d18O": obs_prcp_d18O_df,
    "temp": obs_temp_df,
    "prcp": obs_prcp_df,
    "sam" : sam[((y==2003)&(m>1))+((y==2004)&(m==1))]         ,
},orient="columns")

In [19]:
df_obs = df_obs.dropna(how="any")
num_obs = np.zeros(13)
for mm in range(12):
    num_obs[mm] = len(df_obs[df_obs["Mon"]==mm+1]) 
num_obs[12] =len(df_obs)    

In [22]:
def std_obs(varin,varmean, var):
    df_out = pd.DataFrame.from_dict({
        "std":np.zeros(12)
    }, orient="columns")
    df_out.index = mons[""][:12]
    for mm in range(12):
        df =  varin[varin["Mon"]==mm+1]   
        df_out["std"][mm] = np.nanstd(df[var]-varmean[mm] )

    return df_out.T

In [26]:
df_std_obs = pd.DataFrame(data=[map(str,map(int,num_obs)),
                             ['{:.2f}'.format(n) for n in (std(df_obs[["Mon","temp"]], "temp").T["std"].values )] ,
                             ['{:.2f}'.format(n) for n in (std(df_obs[["Mon","d18O"]], "d18O").T["std"].values )],
                             ['{:.2f}'.format(n) for n in (std(df_obs[["Mon","prcp"]], "prcp").T["std"].values )],
          ], columns=indexlist,
#                           index=["Number","temp","d18O", "prcp"]).T
    index=["Number","$\mathsf{\Delta SAT}$","$\mathsf{\Delta\delta^{18}O_p}$", "Precipitation"])

display(df_std_obs)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
Number,18,23,31,29,31,30,29,31,29,30,26,26,333
$\mathsf{\Delta SAT}$,2.82,3.79,5.47,4.63,5.20,4.44,6.28,5.05,3.98,5.44,4.84,3.02,12.44
$\mathsf{\Delta\delta^{18}O_p}$,4.42,3.56,5.74,4.29,5.28,4.25,5.52,4.55,4.62,5.11,7.28,4.51,10.85
Precipitation,0.21,0.07,0.08,0.05,0.20,0.13,0.17,0.18,0.07,0.05,0.16,0.47,0.19


In [27]:
prcp_summer_obs = df_obs[(df_obs["Mon"]<=2)+(df_obs["Mon"]>=11)]["prcp"]
prcp_winter_obs = df_obs[(df_obs["Mon"]>=3)+(df_obs["Mon"]<=10)]["prcp"]
stats.mannwhitneyu(prcp_winter_obs, prcp_summer_obs, use_continuity=True, alternative=None)

MannwhitneyuResult(statistic=13685.0, pvalue=0.04328494500402202)

In [28]:
temp_summer_obs = df_obs[(df_obs["Mon"]<=2)+(df_obs["Mon"]>=10)]["temp"]
temp_winter_obs = df_obs[(df_obs["Mon"]>=3)+(df_obs["Mon"]<=9)]["temp"]
stats.bartlett(temp_winter_obs, temp_summer_obs)

BartlettResult(statistic=3.5056919102631254, pvalue=0.06115829403383749)

In [29]:
d18O_summer_obs = df_obs[(df_obs["Mon"]<=2)+(df_obs["Mon"]>=10)]["d18O"]
d18O_winter_obs = df_obs[(df_obs["Mon"]>=3)+(df_obs["Mon"]<=9)]["d18O"]
stats.bartlett(d18O_winter_obs, d18O_summer_obs)

BartlettResult(statistic=8.943717868749152, pvalue=0.002784254497742505)

## Non-parametric tests

In [119]:
p_value=0.05

In [30]:
def spearmanr(varin, var1,var2, p_value=0.05):
    # https://qiita.com/dacciinfo/items/88debe69f9f4e927aafc
    df_out = pd.DataFrame.from_dict({
        "r":np.zeros(12),
        "p":np.zeros(12)
    }, orient="columns")
    df_out.index = mons[""][:12]
    
    for mm in range(12):
        df =  varin[varin["Mon"]==mm+1]   
        df_out["r"][mm], df_out["p"][mm] = stats.spearmanr(df[var1],df[var2])
    return df_out.T

In [44]:
p_value=0.05

In [51]:
df_spearmanr = pd.DataFrame(
    data=[map(str,map(int,num[:12])),
          ['{:.2f}'.format(n) for n in (spearmanr(df_all[["Mon","d18O","temp"]],"temp","d18O").T["r"].values )],
          spearmanr(df_all[["Mon","d18O","temp"]],"temp","d18O").T["p"].values,
          ['{:.2f}'.format(n) for n in (spearmanr(df_all[["Mon","d18O","prcp"]],"prcp","d18O").T["r"].values )],
          spearmanr(df_all[["Mon","d18O","prcp"]],"prcp","d18O").T["p"].values,
          ['{:.2f}'.format(n) for n in (spearmanr(df_all[["Mon","d18O","sam" ]],"sam" ,"d18O").T["r"].values )],
          spearmanr(df_all[["Mon","d18O","sam" ]],"sam" ,"d18O").T["p"].values
    ],
    index=["Number","RSAT","pSAT", "RP", "pP", "RSAM", "pSAM"],
    columns=indexlist[:12]).T

df_spearmanr["RSAT"][df_spearmanr["pSAT"]>p_value] = np.nan
df_spearmanr["RP"][df_spearmanr["pP"]>p_value] = np.nan
df_spearmanr["RSAM" ][df_spearmanr["pSAM" ]>p_value] = np.nan
df_spearmanr["pSAT"] = ['{:.2E}'.format(n) for n in (df_spearmanr["pSAT"])]
df_spearmanr["pP"] = ['{:.2E}'.format(n) for n in (df_spearmanr["pP"])]
df_spearmanr["pSAM" ] = ['{:.2E}'.format(n) for n in (df_spearmanr["pSAM" ])]

display(df_spearmanr.T)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Number,930,847,930,900,926,896,922,923,884,853,893,930
RSAT,NaN,0.59,0.53,0.43,0.49,0.54,0.56,0.51,0.47,0.60,0.38,NaN
pSAT,3.80E-01,5.22E-80,3.64E-67,2.40E-42,3.08E-57,5.53E-69,1.66E-78,3.21E-62,1.25E-49,1.38E-83,1.24E-31,8.46E-01
RP,-0.42,-0.12,0.40,0.56,0.45,0.51,0.47,0.44,0.46,0.25,-0.27,-0.52
pP,5.15E-42,6.32E-04,2.43E-36,9.58E-77,5.55E-47,2.09E-59,5.95E-51,4.48E-46,1.47E-46,5.18E-14,1.60E-16,9.61E-65
RSAM,NaN,-0.10,-0.16,-0.18,-0.26,-0.34,-0.48,-0.39,-0.33,-0.29,-0.16,-0.09
pSAM,1.01E-01,4.59E-03,1.16E-06,5.96E-08,3.11E-16,3.10E-26,6.43E-55,9.37E-36,1.43E-24,2.51E-18,1.35E-06,5.00E-03


In [52]:
df_spearmanr = pd.DataFrame(
    data=[map(str,map(int,num_obs)),
          ['{:.2f}'.format(n) for n in (spearmanr(df_obs[["Mon","d18O","temp"]],"temp","d18O").T["r"].values )],
          spearmanr(df_obs[["Mon","d18O","temp"]],"temp","d18O").T["p"].values,
          ['{:.2f}'.format(n) for n in (spearmanr(df_obs[["Mon","d18O","prcp"]],"prcp","d18O").T["r"].values )],
          spearmanr(df_obs[["Mon","d18O","prcp"]],"prcp","d18O").T["p"].values,
          ['{:.2f}'.format(n) for n in (spearmanr(df_obs[["Mon","d18O","sam" ]],"sam" ,"d18O").T["r"].values )],
          spearmanr(df_obs[["Mon","d18O","sam" ]],"sam" ,"d18O").T["p"].values
    ],
    index=["Number","$R_\mathsf{{\Delta SAT}}$","$p_\mathsf{{\Delta SAT}}$", "$R_\mathsf{{P}}$", "$p_\mathsf{{P}}$", "$R_\mathsf{{SAM}}$", "$p_\mathsf{{SAM}}$"],
    columns=mons[""][:12]).T

df_spearmanr["$R_\mathsf{{\Delta SAT}}$"][df_spearmanr["$p_\mathsf{{\Delta SAT}}$"]>p_value] = np.nan
df_spearmanr["$R_\mathsf{{P}}$"][df_spearmanr["$p_\mathsf{{P}}$"]>p_value] = np.nan
df_spearmanr["$R_\mathsf{{SAM}}$" ][df_spearmanr["$p_\mathsf{{SAM}}$" ]>p_value] = np.nan
df_spearmanr["$p_\mathsf{{\Delta SAT}}$"] = ['{:.2E}'.format(n) for n in (df_spearmanr["$p_\mathsf{{\Delta SAT}}$"])]
df_spearmanr["$p_\mathsf{{P}}$"] = ['{:.2E}'.format(n) for n in (df_spearmanr["$p_\mathsf{{P}}$"])]
df_spearmanr["$p_\mathsf{{SAM}}$" ] = ['{:.2E}'.format(n) for n in (df_spearmanr["$p_\mathsf{{SAM}}$" ])]
display(df_spearmanr.T)

ValueError: 12 columns passed, passed data had 13 columns

In [54]:
def draw_d18O2(df):
    x=np.arange(1,13,1)
    ax  = fig.add_subplot(1,1,1)
    ax.plot(    x,   df["$R_\mathsf{{\Delta SAT}}$"],label="Surface Air Temperature Anomaly",
            linewidth=1 ,                  color="red")
    ax.scatter( x,   df["$R_\mathsf{{\Delta SAT}}$"],\
               marker="o"   ,linewidths=2, c="red"      ,edgecolor="red")
    ax.plot(    x,   df["$R_\mathsf{{P}}$"],label="Daily Precipitation",
            linewidth=1 ,                  color="darkgreen")
    ax.scatter( x,   df["$R_\mathsf{{P}}$"],\
               marker="o"   ,linewidths=2, c="darkgreen",edgecolor="darkgreen")
    ax.plot(    x,   df["$R_\mathsf{{SAM}}$" ],label="SAM-index"  ,
            linewidth=1 ,                  color="blue")
    ax.scatter( x,   df["$R_\mathsf{{SAM}}$"],\
               marker="o"   ,linewidths=2, c="blue"     ,edgecolor="blue")
    #
    ax.set_xticks(x)
    ax.set_xticklabels(mons[""][:12])
#    ax.set_ylim( (-0.6,0.6))
    ax.set_yticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
    ax.axhline(y= 0.0,ls='-' ,linewidth=0.5, color='k')
    ax.axhline(y=-0.2,ls='--',linewidth=0.5, color='k')
    ax.axhline(y=-0.4,ls='--',linewidth=0.5, color='k')
    ax.axhline(y=-0.6,ls='--',linewidth=0.5, color='k')
    ax.axhline(y=-0.8,ls='--',linewidth=0.5, color='k')
    ax.axhline(y= 0.2,ls='--',linewidth=0.5, color='k')
    ax.axhline(y= 0.4,ls='--',linewidth=0.5, color='k')
    ax.axhline(y= 0.6,ls='--',linewidth=0.5, color='k')
    ax.axhline(y= 0.8,ls='--',linewidth=0.5, color='k')
    ax.legend(loc="center")
    #
    ax.set_ylabel("Correlation coefficient")

In [55]:
df_spearmanr_obs = pd.DataFrame(
    data=[num,
          spearmanr(df_obs[["Mon","d18O","temp"]],"temp","d18O").T["r"].values,
          spearmanr(df_obs[["Mon","d18O","temp"]],"temp","d18O").T["p"].values,
          spearmanr(df_obs[["Mon","d18O","prcp"]],"prcp","d18O").T["r"].values,
          spearmanr(df_obs[["Mon","d18O","prcp"]],"prcp","d18O").T["p"].values,
          spearmanr(df_obs[["Mon","d18O","sam" ]],"sam" ,"d18O").T["r"].values,
          spearmanr(df_obs[["Mon","d18O","sam" ]],"sam" ,"d18O").T["p"].values
         ],
    index=["Number","$R_\mathsf{{\Delta SAT}}$","$p_\mathsf{{\Delta SAT}}$", "$R_\mathsf{{P}}$", "$p_\mathsf{{P}}$", "$R_\mathsf{{SAM}}$", "$p_\mathsf{{SAM}}$"],
    columns=mons[""][:12]).T

df_spearmanr_obs["$R_\mathsf{{\Delta SAT}}$"][df_spearmanr_obs["$p_\mathsf{{\Delta SAT}}$"]>p_value] = np.nan
df_spearmanr_obs["$R_\mathsf{{P}}$"][df_spearmanr_obs["$p_\mathsf{{P}}$"]>p_value] = np.nan
df_spearmanr_obs["$R_\mathsf{{SAM}}$" ][df_spearmanr_obs["$p_\mathsf{{SAM}}$" ]>p_value] = np.nan
#
fig = plt.figure(figsize=(10,5)) 
ax = draw_d18O2(df_spearmanr_obs)
plt.title("Spearman's correlation coefficients with $\mathsf{\delta^{18}O_p}$ anomaly on every months @Dome Fuji")
plt.show()

ValueError: 12 columns passed, passed data had 13 columns

Kanon Kino (kanon[at]aori.u-tokyo.ac.jp)